<a href="https://colab.research.google.com/github/Rhriti/Predii/blob/gh-pages/LLM_NER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install -U spacy -q


In [ ]:
import spacy
from spacy.tokens import DocBin
from tqdm import  tqdm

nlp=spacy.blank("en")
db=DocBin()

In [ ]:
import json
f=open('./final_annotations.json')
TRAIN_DATA=json.load(f)

In [ ]:

for ele in tqdm(TRAIN_DATA['annotations']):
    if ele is None:continue
    text=ele[0]
    annot=ele[1]
    doc = nlp.make_doc(text)
    ents = []
    for start, end, label in annot["entities"]:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    doc.ents = ents
    db.add(doc)

db.to_disk("./training_data.spacy") # save the docbin object

100%|██████████| 48/48 [00:00<00:00, 594.40it/s]


In [ ]:
! python -m spacy init config config.cfg --lang en --pipeline ner --optimize efficiency --force

⚠ To generate a more effective transformer-based config (GPU-only),
install the spacy-transformers package and re-run this command. The config
generated now does not use transformers.
ℹ Generated config template specific for your use case
- Language: en
- Pipeline: ner
- Optimize for: efficiency
- Hardware: CPU
- Transformer: None
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [ ]:
! python -m spacy train config.cfg --output ./ --paths.train ./training_data.spacy --paths.dev ./training_data.spacy

ℹ Saving to output directory: .
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     76.92    0.00    0.00    0.00    0.00
  4     200        336.66   2703.75   26.33   30.43   23.20    0.26
  8     400       8052.72   1212.98   71.22   76.92   66.30    0.71
 13     600       8928.06    700.48   83.58   90.91   77.35    0.84
 18     800        293.19    521.88   85.71   88.76   82.87    0.86
 23    1000        291.03    477.31   82.32   75.81   90.06    0.82
 29    1200        287.40    466.81   82.72   78.61   87.29    0.83
 37    1400        441.10    646.05   88.69   96.13   82.32    0.89
 46    1600        328.57    748.58 

In [ ]:
nlp_ner = spacy.load("/content/model-best")

In [ ]:
doc=nlp_ner('''A PROPANE LEAK, IN THE PRESENCE OF AN IGNITION SOURCE, COULD RESULT IN A FIRE AND/OR EXPLOSION CAUSING PROPERTY DAMAGE AND/OR PERSONAL INJURY.	DEALERS WILL REPLACE THE PROPANE HOSE WITH ONE OF SHORTER LENGTH AND WILL ALSO SHORTEN THE HOSE SUPPORT TO MOVE THE HOSE AWAY FROM THE TIRE.  THE RECALL BEGAN ON FEBRUARY 1, 2008.  OWNERS MAY CONTACT KEYSTONE RV AT 1-866-425-4369.	KEYSTONE RV RECALL NO. 08-100.CUSTOMERS MAY ALSO CONTACT THE NATIONAL HIGHWAY TRAFFIC SAFETY ADMINISTRATION'S VEHICLE SAFETY HOTLINE AT 1-888-327-4236 (TTY 1-800-424-9153), OR GO TO HTTP://WWW.SAFERCAR.GOV.
''')


In [ ]:
spacy.displacy.render(doc, style="ent", jupyter=True)

In [ ]:
doc3=nlp_ner('''The incorrectly sized fuse may blow during an ABS braking event, increasing the risk of a crash.	Oshkosh will provide owners new fuses, a new fuse label, and installation instructions, free of charge.  Owner notification letters were mailed on July 29, 2021.  Owners may contact Oshkosh customer service at 1-888-686-7278.	Owners may also contact the National Highway Traffic Safety Administration Vehicle Safety Hotline at 1-888-327-4236 (TTY 1-800-424-9153), or go to www.nhtsa.gov.
''')
spacy.displacy.render(doc3, style="ent", jupyter=True)

*NER heavily rely on text data for contextual purpose since there is no context of vehicle model in training data it woule be able to predict vehicle name in many cases or in almost all cases apart from training data*

*The effectiveness of an NER model comes from understanding how entities relate to the context of the surrounding words. For example, "Ford" might be a vehicle model, but it could also appear as part of an organization name or in a different context. The model needs context to distinguish between these possibilities*

In [ ]:
import spacy

# Assuming your model is stored in the current directory
model_directory = './output'

# Load the trained model
nlp = spacy.load(model_directory)

# Save the model to a directory for future use
nlp.to_disk('path_to_save_model')

print(f'Model saved to path_to_save_model')
